In [69]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd  

In [70]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [71]:
def id_split(s):
    s = s + '000'
    s = re.sub('[^\d]', '', s)  
    return s[:4] 
#print( id_split('5.1-4') )

In [72]:
def id_num(s):
    if s == '':
        return 0
    s = s.strip().lower().replace(' ', '')
    l = ['a','b','c','d','e','f','g','h']
    d = [.100, 1, 100, 10000]
    v = [0,0,0,0]
    for i, c in enumerate(l):
        s = s.replace(c,f'{i}') 
    l = s.split('.')
    for i, c in enumerate(l):
        v[i] =  int(c) / d[i] 
    return sum(v)  
print(id_num('1.1.1')) 
print(id_num('1.1.10.1')) 

11.01
11.1001


In [73]:
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df[cols] = df[cols].astype(str)
    df['IDNUM'] = df['IDTEXT'].apply(id_num)
    df.sort_values('IDNUM',inplace=True)   
    df['QPK'] = range(28200, len(df)+28200 )  
    df['sortpos'] = range(1, len(df)+1)  
    return df

In [18]:
query = f""" \
    SELECT TOP 5000 
      IDTEXT AS IDTEXT 
    , PK_Question
    , PK_Question as [2021PK_Question]
    , QGROUP
    , '@FormName' AS  FormName
    , '@PK_QGroup' AS  FK_QuestionGroup
    , sortpos AS  sortpos 
    , NULL AS  FK_Question_Picklist
    , CONVERT(NVARCHAR(5), FK_PickListType) AS  FK_PickList 
    , ISNULL(FK_QuestionType, '') AS  FK_QuestionType
    , NULL AS  FK_InputType
    , CONVERT(NVARCHAR(5), FK_PickList) AS FK_PickListType
    , NULL AS  WarningQuestion 
    , NULL AS  ExternalLinkType
    , NULL AS  PK_ExternalLink
    , ISNULL(QText, '') AS  QuestionText
    , ISNULL(help_text, '') AS  help_text 
    FROM vwQuestions WHERE PK_FORM LIKE '2021-A-HVA'  
"""  
dfq  = get_df(query) 
dfq = normalizedf(dfq)
dfq.sort_values(['QGROUP','IDNUM'], inplace=True)
dfq.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfq.csv', index=False)  
dfq[:2] 

,IDTEXT,PK_Question,2021PK_Question,QGROUP,FormName,FK_QuestionGroup,sortpos,FK_Question_Picklist,FK_PickList,FK_QuestionType,FK_InputType,FK_PickListType,WarningQuestion,ExternalLinkType,PK_ExternalLink,QuestionText,help_text,IDNUM,QPK
0,0.1,22575,22575,2431,@FormName,@PK_QGroup,14,None,None,18,None,None,None,None,None,HVA Information,,1.0,28213
2,1,22577,22577,2431,@FormName,@PK_QGroup,15,None,None,9,None,None,None,None,None,What is the purpose of the HVA?,,10.0,28214


In [19]:
dfh = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='1') 
dfh = normalizedf(dfh) 
dfh['QuestionText'] = ''
dfh[0:2] 

,IDTEXT,FK_QuestionType,FK_PickListType,IDNUM,QPK,sortpos,QuestionText
0,1,9,nan,10.0,28200,1,
1,1.1,17,nan,11.0,28201,2,


In [20]:
df1 = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='ALL') 
df1 = normalizedf(df1) 
df1[1:2]  # .loc[ df1['IDTEXT'].str.contains('5.') ]

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos
1,1,What is the purpose of the HVA?,2431.0,10.0,28201,2


ppl = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList' )
l=[]
c=0
for i,r in ppl.iterrows():
    if i % 2==1: 
        continue
    c=c+1
    IDTEXT = f"1.1.{ c }"
    QuestionText = f"{ r['DisplayValue'] }"
    l.append( {'IDTEXT':f'{IDTEXT}', 'QuestionText': f"If {QuestionText}, describe:"})
ppl = pd.DataFrame(l)  
ppl.to_csv(r'C:\Users\Tim\Documents\1802\HVA\ppl.csv', index=False)  
ppl[0:2]


In [66]:
dfp = pd.read_csv(r'C:\Users\Tim\Documents\1802\HVA\HVAPurpose.csv' )
dfp['IDTEXT'] = dfp['IDTEXT'].apply(lambda s : s.replace('.a','.1'))
dfp['IDTEXT'] = dfp['IDTEXT'].apply(lambda s : s.replace('.b','.2'))
dfp = normalizedf(dfp)  
dfp[:4]

,QuestionText,IDTEXT,IDNUM,QPK,sortpos
0,.GOV Domain Registration Systems,1.1.1,11.01,28200,1
1,Access to Data systems,1.1.2,11.02,28201,2
2,Agency support tool,1.1.3,11.03,28202,3
3,Business Intelligence Services,1.1.4,11.04,28203,4


In [23]:
dfm = pd.merge(df1, dfp, how='outer', on=['IDTEXT', 'QuestionText'], suffixes=(None, '__')) 
dfm = normalizedf(dfm)
dfm.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfm.csv', index=False)  
dfm[0:2]  

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos
0,0.1,HVA Information,nan,1.0,28200,1
1,1,What is the purpose of the HVA?,2431.0,10.0,28201,2


In [67]:
dfa = dfm
dfa = normalizedf(dfa) 
for i,r in dfa.iterrows():
    v=dfa.loc[i, 'QuestionText']
    if v=='':
        dfa.loc[i, 'QuestionText']=dfa.loc[i, 'QuestionText_']
dfa.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfa.csv', index=False)   
dfa['QuestionText'].fillna('')
dfa[0:2] 

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos
0,0.1,HVA Information,nan,1.0,28200,1
1,1,What is the purpose of the HVA?,2431.0,10.0,28201,2


In [68]:
dff = pd.merge(dfa, dfq, how='left', on='IDTEXT', suffixes=('', '_y')) 
dff['FK_QuestionType'].fillna('9', inplace=True)
dff['FK_PickList'].fillna('NULL', inplace=True)
dff['FK_PickListType'].fillna('NULL', inplace=True)
dff['WarningQuestion'].fillna('NULL', inplace=True)
dff['PK_ExternalLink'].fillna('NULL', inplace=True)
dff['PK_Question']=dff['QPK']
dff['FK_InputType'].fillna('NULL', inplace=True) 
dff['help_text'].fillna('NULL', inplace=True) 
dff = dff.fillna('NULL')
dff  = dff.replace('None','NULL')
dff.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfa.csv', index=False)   
dff[150:]

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos,PK_Question,2021PK_Question,QGROUP_y,FormName,...,FK_QuestionType,FK_InputType,FK_PickListType,WarningQuestion,ExternalLinkType,PK_ExternalLink,QuestionText_y,help_text,IDNUM_y,QPK_y
150,20.3,Does D/A for this HVA have an ISA/MOU with own...,nan,203.0,28350,151,28350,NULL,NULL,NULL,...,9,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
151,20.4,Is this connected system Internal to your Agency?,nan,204.0,28351,152,28351,NULL,NULL,NULL,...,9,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
152,20.5,"If 'Yes' to Internal to Agency, then Is this s...",nan,205.0,28352,153,28352,NULL,NULL,NULL,...,9,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
153,20.6,Is this interconnection/data exchange mission ...,nan,206.0,28353,154,28353,NULL,NULL,NULL,...,9,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
154,20.7,Is this interconnection,nan,207.0,28354,155,28354,NULL,NULL,NULL,...,9,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
155,21,Please list each direct interconnection or dat...,nan,210.0,28355,156,28355,22744,2435,@FormName,...,69,NULL,NULL,NULL,NULL,NULL,Please list each direct interconnection or dat...,,210.0,28296.0
156,21.1,Name of System,nan,211.0,28356,157,28356,22745,2435,@FormName,...,18,NULL,NULL,NULL,NULL,NULL,Name of Connected System,,211.0,28297.0
157,21.2,Please enter the Name of the organization that...,nan,212.0,28357,158,28357,22746,2435,@FormName,...,18,NULL,NULL,NULL,NULL,NULL,Is this connected system Internal to your Agency?,,212.0,28298.0
158,21.3,Please indicate the organization type for the ...,nan,213.0,28358,159,28358,22747,2435,@FormName,...,18,NULL,NULL,NULL,NULL,NULL,If 'Yes' to Internal to Agency then Is this sy...,,213.0,28299.0
159,21.4,Does the D/A for this HVA have an ISA/MOU with...,nan,214.0,28359,160,28359,22748,2435,@FormName,...,18,NULL,NULL,NULL,NULL,NULL,Is this interconnection/data exchange mission ...,,214.0,28300.0


In [60]:
query = f"""  SELECT TOP 1 
    PK_Question, FormName, FK_QuestionGroup,sortpos,identifier_text, FK_QuestionType
    , FK_InputType, FK_PickListType, WarningQuestion, ExternalLinkType, QuestionText, help_text 
    FROM fsma_Questions 
    """  
df_qmeta = get_df(query) 
cols = ','.join(df_qmeta.columns)
INS = f'INSERT INTO fsma_Questions ({cols}) VALUES '
print(INS)

INSERT INTO fsma_Questions (PK_Question,FormName,FK_QuestionGroup,sortpos,identifier_text,FK_QuestionType,FK_InputType,FK_PickListType,WarningQuestion,ExternalLinkType,QuestionText,help_text) VALUES 


In [116]:
dff = get_df("""
SELECT PK_Question QPK
  , CONVERT(NVARCHAR(9), FK_QuestionType) + '' FK_QuestionType
  , CONVERT(NVARCHAR(9), FK_PickListType) + '' FK_PickListType
  , QTEXT As QuestionText
  , help_text
  , IdText as IDTEXT 
  FROM vwQuestions WHERE QGROUP=2433
  ORDER BY SortPos ASC
""") 
dff=dff.fillna('~')
dff

,QPK,FK_QuestionType,FK_PickListType,QuestionText,help_text,IDTEXT
0,22693,63,~,Modernization,~,
1,22850,63,~,12.0 Does this system directly employ or opera...,~,12
2,22851,17,~,"Adoption of application, data, and enterprise ...",~,12.1
3,22852,17,~,Multi-Factor Authentication,~,12.2
4,22853,17,~,Least Privilege,~,12.3
5,22854,17,~,Application Whitelisting,~,12.4
6,22855,17,~,Mobile Device Management,~,12.5
7,22856,17,~,Cloud-oriented protection (If in a cloud deplo...,~,12.6
8,22857,17,~,Adoption of automation and outcome-oriented se...,~,12.7
9,22858,17,~,Utilization of shared security services such a...,~,12.8


In [120]:
sql = "" 
for i, r in dff[:].iterrows(): 
    s = "(@QID, @FormName, @PK_QGroup, @SORD, N'@IDTEXT', @QTYPE, 1, @PKTYPE, 0, NULL, N'@QTEXT','@HTEXT'),"
    s = s.replace('@QID',  str(r['QPK']))
    s = s.replace('@SORD', str(i+1)) 
    s = s.replace('@QTYPE', r['FK_QuestionType'])
    s = s.replace('@PKTYPE', r['FK_PickListType'])
    s = s.replace('@IDTEXT', r['IDTEXT']) 
    s = s.replace('@QTEXT', r['QuestionText'].replace('\'', '`').replace('\ufffd', ''))  
    s = s.replace('@HTEXT', r['help_text'])  
    sql = sql + s + '\n'
sql = sql.replace("'~')", "NULL)")    
# sql = sql.replace("N'NULL'", "N''")   
sql = sql.replace("~,", "NULL,")  
# sql = f'{INS}\n {sql}\n' 
print( sql )

(22693, @FormName, @PK_QGroup, 1, N'', 63, 1, NULL, 0, NULL, N'Modernization',NULL),
(22850, @FormName, @PK_QGroup, 2, N'12', 63, 1, NULL, 0, NULL, N'12.0 Does this system directly employ or operate in an environment that has adopted any of the following cybersecurity mitigation technologies?',NULL),
(22851, @FormName, @PK_QGroup, 3, N'12.1', 17, 1, NULL, 0, NULL, N'Adoption of application, data, and enterprise level protections such as Encrypting Data at Rest and Encrypting Data in Transit ',NULL),
(22852, @FormName, @PK_QGroup, 4, N'12.2', 17, 1, NULL, 0, NULL, N'Multi-Factor Authentication',NULL),
(22853, @FormName, @PK_QGroup, 5, N'12.3', 17, 1, NULL, 0, NULL, N'Least Privilege',NULL),
(22854, @FormName, @PK_QGroup, 6, N'12.4', 17, 1, NULL, 0, NULL, N'Application Whitelisting',NULL),
(22855, @FormName, @PK_QGroup, 7, N'12.5', 17, 1, NULL, 0, NULL, N'Mobile Device Management',NULL),
(22856, @FormName, @PK_QGroup, 8, N'12.6', 17, 1, NULL, 0, NULL, N'Cloud-oriented protection (If in a

In [ ]:
with open(fr'C:\Users\Tim\Documents\1802\HVA\update.sql', 'w') as f:
    f.write(sql)  



df = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList').fillna('NULL') 
cols = list(df) # Creates list of all column headers
df[cols] = df[cols].astype(str)
df['DisplayCode'] =  df['DisplayValue'].apply(lambda s: re.sub('[^A-Z0-9]','',s.upper()))
df['DisplayCode'] =  df['DisplayCode'].apply(lambda s: re.sub('[AEIOURST]','',s)[:8])
s =''
for i,r, in df.iterrows():
    PK_PickList=r['PK_PickList']
    DisplayCode=r['DisplayCode']
    DisplayValue=r['DisplayValue'] 
    if DisplayValue=='Other':
        DisplayCode='OTH'
    s = s + f"({PK_PickList},@PKT, N'{DisplayCode}',N'{DisplayValue}',{i}, GETDATE(), 1),\n"

print(s)


In [77]:
query = f""" \
    SELECT TOP 5000 
      IDTEXT AS IDTEXT 
    , PK_Question 
    FROM vwQuestions WHERE PK_FORM LIKE '2022-A-HVA'  
"""  
dfdep = get_df(query)  
# dfdep.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfdep.csv', index=False)  
dfdep[:] 

,IDTEXT,PK_Question
0,0.1,28200
1,1,28201
2,1.1,28202
3,2,28228
4,2.1,28229
...,...,...
103,1.1.45,28334
104,1.1.46,28335
105,1.1.47,28336
106,1.1.48,28337


In [92]:
INS = f"""  INSERT INTO [fsma_QuestionDependencies] 
(FK_Question, FK_Question_Master, Question_Master_CodeValue_ToMakeRequired) VALUES 
    """  
sql = ""   
for i, r in dfdep[:].iterrows(): 
    mast = 28211
    s = f"({r['PK_Question']}, {mast}, 'Y') ,"     
    sql = sql + s + '\n' 
sql = f'{INS}\n {sql}\n' 
# print(sql)